In [44]:
DB_HOST = "localhost"
DB_PORT = "3306"
DB_USER = "root"
DB_PASSWORD = "urubu100"
DATABASE = "ALGAS"

CREATE_QUERY = "CREATE TABLE IF NOT EXISTS dados (id INT PRIMARY KEY AUTO_INCREMENT, valor INT, espacoMemoria INT, tempoExecucao INT)"
INSERT_QUERY = "INSERT INTO dados (id, valor, espacoMemoria, tempoExecucao) VALUES (default, %s, %s, %s)"
SELECT_QUERY = "SELECT id, valor, espacoMemoria, tempoExecucao FROM dados"
DELETE_QUERY = "TRUNCATE dados"

DROP_QUERY = "DROP TABLE dados"

In [45]:
from mysql.connector import connect as connect_my
def get_connection():
    return connect_my(
        # host = DB_HOST + ":" + DB_PORT,
        host = DB_HOST,
        user = DB_USER,
        password = DB_PASSWORD,
        database = DATABASE
        )

def create_db():
    con = get_connection()
    c = con.cursor()
    c.execute(CREATE_QUERY)    
    return con

In [46]:
def get_insert_query():
    return INSERT_QUERY

def insert_data(self, data: list):
    con = get_connection()
    c = con.cursor()
    
    insert = get_insert_query()

    for value in data:
        c.execute(insert, value)

    con.commit()

def select_data():
    con = get_connection()
    c = con.cursor()
    
    c.execute(SELECT_QUERY)
    
    return c.fetchall()

In [47]:
class DatabaseManager:
    def __init__(self):
        self.connection = self.create()
        self.cursor = None
    
    def create(self):
        return get_connection()
    
    def transact_begin(self):
        self.cursor = self.connection.cursor()
    
    def transact_commit(self):
        self.connection.commit()

    def insert(self, dados: list):
        insert_data(self.connection, dados)
        
    def select(self) -> list:
        return select_data(cursor=self.connection.cursor())


In [48]:
from datetime import datetime
from sys import getsizeof

def append_data(data) -> list:
    dados = []
    memory_accumulator = 0
    for range_data in data.values():
        for value in range_data:
            inicio = datetime.now()
            acumulador = 0
            for i in range(value, 0, -1):
                acumulador += i
            memory_accumulator += getsizeof(value)
            duracao = datetime.now() - inicio
            dados.append((value, memory_accumulator, duracao.microseconds))
    return dados


def generate_data(data : dict) -> list:
    dados = append_data(data)
    return dados 

In [54]:
rangeDadosInicio=0
rangeDadosFim=0
rangeDadosPulo=0
rangeDados={}
while True:
    print("Menu")
    print("1-Usuario, por favor insira range de dados 5 vezes:")
    print("2-sair")
    print("3-Inserir dados(somente depois do opcao 1)")
    answer = int(input("escolha:"))
    if answer == 2:
        break
    elif answer == 1:
        for i in range(0,5,1):
            print('range: ',i)
            rangeDadosInicio = int(input("insira o range inicio:"))
            rangeDadosFim = int(input("insira range fim:"))
            rangeDadosPulo = int(input("insira range pulo:"))
            rangeDados[i] = [rangeDadosInicio,rangeDadosFim,rangeDadosPulo]
        print('range: ', end='')
        doubleRange = {k:v for (k,v) in rangeDados.items()}
        print(doubleRange)
    elif answer == 3:        
        dadosX = generate_data(rangeDados)
        for i in dadosX:
            print('inserindo:',i)
        db = DatabaseManager()
        db.transact_begin()
        db.insert(dadosX)
        db.transact_commit()
        print("Inseriu")

Menu
1-Usuario, por favor insira range de dados 5 vezes:
2-sair
3-Inserir dados(somente depois do opcao 1)
escolha:1
range:  0
insira o range inicio:10
insira range fim:60
insira range pulo:1
range:  1
insira o range inicio:1000
insira range fim:6000
insira range pulo:100
range:  2
insira o range inicio:100000
insira range fim:600000
insira range pulo:100000
range:  3
insira o range inicio:10000
insira range fim:60000
insira range pulo:1000
range:  4
insira o range inicio:100
insira range fim:600
insira range pulo:10
range: {0: [10, 60, 1], 1: [1000, 6000, 100], 2: [100000, 600000, 100000], 3: [10000, 60000, 1000], 4: [100, 600, 10]}
Menu
1-Usuario, por favor insira range de dados 5 vezes:
2-sair
3-Inserir dados(somente depois do opcao 1)
escolha:3
inserindo: (10, 28, 0)
inserindo: (60, 56, 0)
inserindo: (1, 84, 0)
inserindo: (1000, 112, 0)
inserindo: (6000, 140, 2339)
inserindo: (100, 168, 0)
inserindo: (100000, 196, 10559)
inserindo: (600000, 224, 48028)
inserindo: (100000, 252, 0)
i